In [1]:
import numpy as np
import pandas as pd
import librosa
import os
from glob import glob
from tqdm import tqdm
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
from sklearn.metrics import f1_score

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
max_length = 17640

X_train_pad_seq = np.load('./X_train_pad_seq_m3.npy')
X_test_pad_seq = np.load('X_test_pad_seq_m3.npy')

agumented_train_label = np.load('agumented_train_label.npy')
y_test = np.load('y_test.npy')

In [5]:
sample_rate = 22050
def convert_to_spectrogram(raw_data):
    '''converting to spectrogram'''
    spectrum = librosa.feature.melspectrogram(y=raw_data, sr=sample_rate, n_mels=64)
    logmel_spectrum = librosa.power_to_db(S=spectrum, ref=np.max)
    return logmel_spectrum

In [6]:
X_train_spectrogram = np.array([ convert_to_spectrogram(x) for x in tqdm(X_train_pad_seq)])
X_test_spectrogram = np.array([ convert_to_spectrogram(x) for x in tqdm(X_test_pad_seq)])


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14400/14400 [01:14<00:00, 193.06it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:02<00:00, 191.74it/s]


In [7]:
class F1ScoreCB(tf.keras.callbacks.Callback):
    
    def __init__(self, train_data, test_data):
        super().__init__()
        self.train_data = train_data
        self.test_data = test_data
        self.history = {}
        self.history['val_f1_score'] = []
  
    def on_epoch_end(self, epochs, logs = {}):
        train_preds = np.argmax(self.model.predict(self.train_data[0]), axis = -1)
        train_f1_score = f1_score(self.train_data[1], train_preds, average='micro')
        train_f1_score = np.round(train_f1_score, 4)

        test_preds = np.argmax(self.model.predict(self.test_data[0]), axis = -1)
        test_f1_score = f1_score(self.test_data[1], test_preds, average='micro')
        test_f1_score = np.round(test_f1_score, 4)
        self.history['val_f1_score'].append(test_f1_score)

        print(f"train_f1_score: {train_f1_score} - val_f1_score: {test_f1_score}")
        print("########### EPOCH ENDED ##########")

In [8]:
input_layer_m4 = Input(shape=(64,35,),name="Input_spectogram")
lstm_layer_m4 = LSTM(units = 256,return_sequences = True
                     ,name="LSTM_M4")(input_layer_m4)
avg_pooled = tf.reduce_mean(lstm_layer_m4 ,axis=-1)
flatten_m4 = tf.keras.layers.Flatten(name='Flatten_M4')(avg_pooled)
dense_layer_m4 = Dense(256,activation='relu')(flatten_m4)
output_m4 = Dense(10,activation = 'softmax')(dense_layer_m4)
model_4 = Model(input_layer_m4,output_m4)
model_4.summary()

2022-10-29 00:23:50.869246: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-29 00:23:51.405390: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9651 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:82:00.0, compute capability: 7.5


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_spectogram (InputLaye  [(None, 64, 35)]         0         
 r)                                                              
                                                                 
 LSTM_M4 (LSTM)              (None, 64, 256)           299008    
                                                                 
 tf.math.reduce_mean (TFOpLa  (None, 64)               0         
 mbda)                                                           
                                                                 
 Flatten_M4 (Flatten)        (None, 64)                0         
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dense_1 (Dense)             (None, 10)                2570  

In [11]:
model_4.compile(optimizer = tf.keras.optimizers.Adam(0.0001), loss = 'sparse_categorical_crossentropy')
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.005, patience=3, verbose=1)

model_4.fit(X_train_spectrogram, agumented_train_label,
           validation_data = (X_test_spectrogram, y_test),
           batch_size = 128, epochs = 50,
            callbacks = [F1ScoreCB((X_train_spectrogram, agumented_train_label), (X_test_spectrogram, y_test)),early_stop]
          )

Epoch 1/50


  0%|                                                                                                                                              | 0/14400 [18:02<?, ?it/s]


13/13 [==============================] - 0s 4ms/step
train_f1_score: 0.8717 - val_f1_score: 0.8575
########### EPOCH ENDED ##########
113/113 [==============================] - 6s 40ms/step - loss: 0.4135 - val_loss: 0.4019
Epoch 2/50
13/13 [==============================] - 0s 4ms/step
train_f1_score: 0.8773 - val_f1_score: 0.875
########### EPOCH ENDED ##########
113/113 [==============================] - 4s 32ms/step - loss: 0.3917 - val_loss: 0.3732
Epoch 3/50
13/13 [==============================] - 0s 4ms/step
train_f1_score: 0.8902 - val_f1_score: 0.8725
########### EPOCH ENDED ##########
113/113 [==============================] - 3s 29ms/step - loss: 0.3739 - val_loss: 0.3690
Epoch 4/50
13/13 [==============================] - 0s 5ms/step
train_f1_score: 0.8873 - val_f1_score: 0.88
########### EPOCH ENDED ##########
113/113 [==============================] - 4s 31ms/step - loss: 0.3539 - val_loss: 0.3836
Epoch 5/50
13/13 [==============================] - 0s 4ms/step
train_f1_s